In [ ]:
import tensorflow as tf


In [ ]:
num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", num_gpus_available)
# assert num_gpus_available > 0


Num GPUs Available:  1


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.2 MB/s eta 0:00:00


In [2]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'transformers'

In [ ]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import tensorflow_datasets as tfds
ds = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

DownloadError: ignored

In [ ]:
df = tfds.as_dataframe(ds)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df["Sentiment"] = df["data/star_rating"].apply(lambda score: "positive" if score >= 3 else "negative")
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})

NameError: ignored

In [ ]:
df['short_review'] =df['data/review_body'].str.decode("utf-8")

In [ ]:
df = df[["short_review", "Sentiment"]]

In [ ]:
#Dropping last n rows using drop
n = 54975
df.drop(df.tail(n).index,
        inplace = True)

<ipython-input-14-aea205828031>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df.tail(n).index,


In [ ]:
index = df.index
number_of_rows = len(index)
print(number_of_rows)

50000


In [ ]:
df.tail()

,short_review,Sentiment
49995,Awesome,1
49996,I got this cover for my daughter's ipod. It is...,1
49997,This battery does not fit the T-mobile G2 is i...,0
49998,"It has good sound but , the charge only last 1...",1
49999,What I imaginaed getting from this machine was...,1


In [ ]:
df.head()

,short_review,Sentiment
0,Does not work,0
1,This is a great wiring kit i used it to set up...,1
2,It works great so much faster than USB charger...,1
3,This product was purchased to hold a monitor o...,1
4,it works but it has really bad sound quality. ...,1


In [ ]:

reviews = df['short_review'].values.tolist()
labels = df['Sentiment'].tolist()

NameError: ignored

In [ ]:
print(reviews[:2])
print(labels[:2])

['Does not work', 'This is a great wiring kit i used it to set up my Pyle 2000 watt amp to 2 alpine subs and worked just fine. im not sure about the power cord and its length because my battery is in the back but the ground cord is a bit short and thats its only flaw and that why i gave it 4 stars. great kit']
[0, 1]


In [ ]:
from sklearn.model_selection import train_test_split
training_sentences, validation_sentences, training_labels, validation_labels = train_test_split(reviews, labels, test_size=.2)

NameError: ignored

In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
tokenizer([training_sentences[0]], truncation=True,
                            padding=True, max_length=128)

NameError: ignored

In [ ]:
  train_encodings = tokenizer(training_sentences,
                              truncation=True,
                              padding=True)
  val_encodings = tokenizer(validation_sentences,
                              truncation=True,
                              padding=True)

NameError: ignored

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    training_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    validation_labels
))

NameError: ignored

In [ ]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])


In [ ]:
model.fit(train_dataset.shuffle(100).batch(16),
          epochs=2,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/2
2500/2500 [==============================] - 2382s 940ms/step - loss: 0.2131 - accuracy: 0.9111 - val_loss: 0.1830 - val_accuracy: 0.9249
Epoch 2/2
2500/2500 [==============================] - 2336s 934ms/step - loss: 0.1375 - accuracy: 0.9463 - val_loss: 0.2253 - val_accuracy: 0.9154


In [1]:
model.save_pretrained("/content/drive/MyDrive/FINE TUNING BERT")

NameError: name 'model' is not defined

In [2]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("/content/drive/MyDrive/FINE TUNING BERT")

NameError: name 'TFDistilBertForSequenceClassification' is not defined

In [1]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("/FINE TUNING BERT/tf_model.h5")

NameError: name 'TFDistilBertForSequenceClassification' is not defined

In [ ]:
test_sentence = "This product is awful. I hate it"



predict_input = tokenizer.encode(test_sentence,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

tf_output = loaded_model.predict(predict_input)[0]


tf_prediction = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive']
label = tf.argmax(tf_prediction, axis=1)
label = label.numpy()
print(labels[label[0]])

NameError: ignored

In [ ]:
# for i in val_encodings:

#   predict_input = i

#   tf_output = loaded_model.predict(predict_input)


#   tf_prediction = tf.nn.softmax(tf_output, axis=1)
#   labels = ['Negative','Positive']
#   label = tf.argmax(tf_prediction, axis=1)
#   label = label.numpy()
#   print(labels[label[0]])

IndexError: ignored